In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import albumentations as A
import albumentations.pytorch
import torchvision
import numpy as np
from PIL import Image

In [ ]:
decoder = torchvision.transforms.ToPILImage()
transforms = A.Compose([
    A.pytorch.ToTensorV2()
])

In [ ]:
origin = Image.open('xichun.jpg').convert("RGB").resize((512, 512))
plt.imshow(origin)
plt.show()
nimage = np.array(origin)

# 将图片转成 float32 格式，因为下面 Tensor 中需要浮点数才能计算
fimage = np.array(origin, dtype=np.float32)
plt.imshow(fimage.astype(np.uint8))
plt.show()

In [ ]:
# 将图片转成浮点张量
# 另外将图片的三个维度重新排列一下，默认图片是 (宽度, 高度, 通道) 
# 转换之后变成了 (通道，高度，宽度)
# 然后再生成一维，变成四维的数据，(批量，通道，高度，宽度)，这是 下文 Conv2d 要求的

input = torch.FloatTensor(fimage).permute(2, 1, 0).unsqueeze(0)
print(input.shape)
t = input.squeeze(0)
print(t.shape)

outputs = [input]

# 将卷积张量转换成图片，并显示
output = t.detach().permute(2, 1, 0).numpy().astype(np.uint8)
plt.imshow(output)
plt.show()

In [ ]:
def show_image(output):
    image = output.squeeze(0)
    plt.imshow(image.detach().permute(2, 1, 0).numpy().astype(np.uint8) / 255.0)
    plt.show()

In [ ]:
# 设置随机数种子可以使每次生成模块的随机数都一致
seed = 6666
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# 生成一个二维卷积，输入三个通道，输出三个通道，分别表示 RGB
conv = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1)
output = conv(input)
outputs.append(output)

print(output.shape)

show_image(output)

In [ ]:
# 逆卷积操作
convtrans = nn.ConvTranspose2d(3, 3, kernel_size=2, stride=2, padding=1)
output = convtrans(outputs[0])
show_image(output)

In [ ]:
bn = nn.BatchNorm2d(3)
output = bn(outputs[-1])
outputs.append(output)
show_image(output)

In [ ]:
relu = nn.ReLU(True)
output = relu(outputs[-2])
show_image(output)
outputs.append(output)

In [ ]:
maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
output = maxpool(outputs[-1])
show_image(output)
outputs.append(output)

In [ ]:
a = torch.cat((outputs[0], outputs[1]), dim=3)
print(a.shape)
show_image(a)

b = torch.cat((outputs[0], outputs[1]), dim=2)
print(b.shape)
show_image(b)

c = torch.cat((outputs[0], outputs[1]), dim=1)
print(c.shape)
# show_image(output4)

a, b = torch.split(c, [3, 3], dim=1)
print(a.shape, a.shape)
show_image(a)
show_image(b)